In [2]:
import timeit
import pandas as pd
import os

ROOTDIR = os.path.normpath("N:\_archive\\test\\trans\soundgasmNET")
df = pd.read_csv(os.path.join(ROOTDIR, "sgasm_rip_db.csv"), sep=";", encoding="utf-8", index_col=0)
df_grp = df.groupby("sgasm_user")

url = "https://soundgasm.net/u/belle_in_the_woods/F4M-The-Take-FdomJOIcountdownteasingwhispers"

In [3]:
%timeit url in df_grp.get_group("belle_in_the_woods")["URLsg"]

The slowest run took 7.66 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 527 µs per loop


In [4]:
%timeit url in df[df["sgasm_user"] == "belle_in_the_woods"]["URLsg"]

100 loops, best of 3: 2.24 ms per loop


In [5]:
def t():
    df.query("sgasm_user == 'belle_in_the_woods'")["URLsg"].isin([url]).any()

In [6]:
%timeit t()

100 loops, best of 3: 2.65 ms per loop


In [7]:
%timeit url in df["URLsg"]

The slowest run took 39.70 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 9.47 µs per loop


In [13]:
urls = ["https://soundgasm.net/u/belle_in_the_woods/F4M-The-Take-FdomJOIcountdownteasingwhispers",
       "https://soundgasm.net/u/belle_in_the_woods/F4F-Should-I-Call-intimateerotic-poetry",
       "https://soundgasm.net/u/belle_in_the_woods/F4M-In-Darkness-FsubErotic-Poetry",
       "https://soundgasm.net/u/belle_in_the_woods/F4M-Cum-Lust-cock-worshiperotic-poetry",
       "https://soundgasm.net/u/belle_in_the_woods/F4M-The-Take-FdomJOIcountdownteasingwhispers"]

def filter_alrdy_downloaded(df, dl_dict, currentusr=None):
    # OLD when passing 2pair tuples, unpack tuples in dl_list into two lists
    # url_list, title = zip(*dl_list)
    # filter dupes
    unique_urls = set(dl_dict)
    if currentusr:
        try:
            duplicate = unique_urls.intersection(df_grp.get_group(currentusr)["URLsg"].values)
        except KeyError:
            logger.info("User '{}' not yet in databas!".format(currentusr))
            duplicate = set()
    else:
        # timeit 1000: 0.19
        duplicate = unique_urls.intersection(df["URLsg"].values)

    # set.symmetric_difference()
    # Return a new set with elements in either the set or other but not both.
    # -> duplicates will get removed from unique_urls
    result = list(unique_urls.symmetric_difference(duplicate))

    return result

In [15]:
%timeit filter_alrdy_downloaded(df, urls)

1000 loops, best of 3: 705 µs per loop


In [14]:
%timeit filter_alrdy_downloaded(df, urls, currentusr="belle_in_the_woods")

1000 loops, best of 3: 488 µs per loop
